# Autocomplete

## O que é um ngram ?

### COFFEE

![Texto Alternativo](images/edge_ngram.png)

### Como fica a Indexação

![Texto Alternativo](images/analyzer.png)

## Configurando Index

In [1]:
mappings = {
  "settings": {
    "analysis": {
      "filter": {
        "autocomplete_filter": {
          "type": "edge_ngram",
          "min_gram": 1,
          "max_gram": 20
        }
      },
      "analyzer": {
        "autocomplete": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "autocomplete_filter"
          ]
        },
        "autocomplete_search": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase"
          ]
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "title": {
        "type": "text",
        "analyzer": "autocomplete",
        "search_analyzer": "autocomplete_search"
      }
    }
  }
}

## Inserindo dados

In [2]:
password='XxZiSgIG'

from elasticsearch import Elasticsearch


# Configurações do Elasticsearch
ELASTICSEARCH_HOST = "http://localhost:9200/"
ELASTICSEARCH_USER = "elastic"  # Usuário padrão
ELASTICSEARCH_PASSWORD = password  # senha 

# Conectar ao Elasticsearch
es = Elasticsearch(
    [ELASTICSEARCH_HOST],
    http_auth=(ELASTICSEARCH_USER, ELASTICSEARCH_PASSWORD),
    verify_certs=False  # Desabilitar a verificação do SSL
)

# Verificar a conexão
if es.ping():
    print("Conexão bem-sucedida ao Elasticsearch!")
else:
    print("Falha ao conectar ao Elasticsearch. Verifique as configurações.")


Conexão bem-sucedida ao Elasticsearch!


/tmp/ipykernel_49652/2922222961.py:12: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


### Criando Index para o autocomplete

In [3]:
index_='cursos-autocomplete'

In [4]:
response = es.indices.create(index=index_, body=mappings)

print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'cursos-autocomplete'}


### Inserindo Dados

#### Carregando dados

In [6]:
import pandas as pd

cursos = pd.read_json('data/alura-cursos-formacoes-dataset.json')

cursos['description'] = cursos['description'].apply(lambda x: x['raw'])

cursos_df = cursos[['id','title','description','type','siteUrl']]

cursos_df.head()

,id,title,description,type,siteUrl
0,DEGREE-828628,Flutter: cursos para automatizar testes,<p>Aprenda a usar e personalizar testes automa...,DEGREE,/formacao-cursos-testes-automatizados-aplicati...
1,DEGREE-909375,Dart: cursos para aprender a linguagem utiliza...,<p>Se você busca uma linguagem para desenvolve...,DEGREE,/formacao-dart-cursos-aprender-linguagem-utili...
2,DEGREE-907699,"CSS avançado: implementando designs com Grid, ...",<p>Se você já conhece <strong>HTML</strong> e ...,DEGREE,/formacao-css-cursos-transformar-designs-grid-...
3,DEGREE-824634,"Angular: construa um Design System com Nx, Mon...","<p>Nesta formação, você vai desenvolver do zer...",DEGREE,/formacao-angular-cursos-construir-design-syst...
4,DEGREE-515205,Certificação AZ-900: Microsoft Azure Fundamentals,<p>O curso <strong>AZ-900 - Microsoft Azure Fu...,DEGREE,/formacao-certificacao-az-900-microsoft-azure-...


#### Inserindo

In [7]:
from elasticsearch.helpers import bulk

In [8]:
actions = [
    {
        "_index": index_,
        "_source": curso
    }
    for curso in cursos_df.to_dict(orient='records')
]

In [9]:
bulk(es, actions)

(2500, [])

## Testando Nosso autocomplete

In [10]:
def autocomplete(es, query, index):
    body = {
        "query": {
            "match": {
                "title": {
                    "query": query,
                    "operator": "and"
                }
            }
        }
    }
    result = es.search(index=index, body=body)
    return [hit["_source"]["title"] for hit in result["hits"]["hits"]]

In [12]:
autocomplete(es, "BIG", index_)

['Google Big Query: consultas avançadas',
 'Google BigQuery: navegando na nuvem',
 'Google BigQuery: manipulação de dados',
 'Google BigQuery: dominando Strings e Data',
 'Google BigQuery: dominando funções lógicas e numéricas',
 'Acesso ao Big Query: manipulação programático linguagens',
 'Big Query ML: trabalhando com modelos de classificação']